In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/NLP/nlp_problem_2

/content/drive/My Drive/NLP/nlp_problem_2


In [3]:
#import library
# !pip install pyvi
# !pip install underthesea
# !pip install eli5
import json
# import spacy
import random
import pyvi
from underthesea import chunk 
import nltk
import sklearn_crfsuite
import eli5
import os
from tqdm import trange, tqdm_notebook
import numpy as np
import glob
from past.builtins import xrange

In [4]:
# tạo class đọc file 
class FileReader(object):
    def __init__(self, file_path, encoder = None):
        self.file_path = file_path
        self.encoder = encoder if encoder != None else 'utf-16le'

    def read(self):
        with open(self.file_path,'rb') as f:
            s = f.read() 
        return s

    def content(self):
        s = self.read() 
        return s.decode(self.encoder)
 


In [5]:
#hàm chuyển word thành features theo cấu trúc mô hình yêu cầu
def word2features(sent, i):
    word = sent[i][0] 
    postag = sent[i][1] 
    # khơi tạo features
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    # nếu i > 0 tức là có từ đứng TRƯỚC nó thì sẽ update feature lại theo cấu trúc sau 
    # thêm các feature của từ đằng TRƯỚC nó vào 
    # ngược lại thêm features['BOS'] = TRUE => có nghĩa nó là từ bắt đầu câu BEGIN OF SENTENCE
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    # nếu i < len(sentence) - 1: có nghĩa là có từ đứng SAU nó 
    # sẽ update thêm feature của từ đứng SAU nó   
    # ngược lại thêm features['EOS'] = TRUE => có nghĩa nó là từ kết thúc câu END OF SENTENCE
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]


In [100]:
JSON_FROM_DOCCANO = ["0-29.json1",
                     "30-59.json1",
                     "75-100.json1",
                    "101-200.json1",
                    "201-230.json1",
                    "231-260.json1"]

PATH_JSON =  "/home/duy/Documents/Duy/NLP/problem_1_nlp/nlp_2/json/"
JSON_FROM_DOCCANO = [ PATH_JSON + item for item in JSON_FROM_DOCCANO ]
JSON_ENTITY = PATH_JSON + "data.json"
TOTAL_LABEL_PATH = PATH_JSON + "label.json" 

In [107]:
TRAINING_DATA = []
LABEL = []
# đọc từ những file json doccano để modify cho phù hợp với data cần xử lý
for path in JSON_FROM_DOCCANO:
    print(path,end=" ") 
    with open(path) as f:
        TEXTS = f.readlines() 
        print("sentences",len(TEXTS))
        for doc in TEXTS:
            doc = json.loads(doc)  
            entities = [(en[0], en[1], en[2]) for en in doc['labels']]
            [LABEL.append(item[2].upper()) for item in entities if item[2].upper() not in LABEL]
            if len(entities) == 0:
                continue 
            training_example = (doc['text'], {"entities": entities})
            TRAINING_DATA.append(training_example) 
        f.close()

# ghi file json, entity cho tiện sử dụng lại 
with open(JSON_ENTITY,'w') as f:
    f.write(json.dumps(TRAINING_DATA))

with open(TOTAL_LABEL_PATH,'w') as f:
#     LABEL = [ item.upper() for item in LABEL ]
    f.write(json.dumps(LABEL))

/home/duy/Documents/Duy/NLP/problem_1_nlp/nlp_2/json/0-29.json1 sentences 241
/home/duy/Documents/Duy/NLP/problem_1_nlp/nlp_2/json/30-59.json1 sentences 202
/home/duy/Documents/Duy/NLP/problem_1_nlp/nlp_2/json/75-100.json1 sentences 216
/home/duy/Documents/Duy/NLP/problem_1_nlp/nlp_2/json/101-200.json1 sentences 739
/home/duy/Documents/Duy/NLP/problem_1_nlp/nlp_2/json/201-230.json1 sentences 221
/home/duy/Documents/Duy/NLP/problem_1_nlp/nlp_2/json/231-260.json1 sentences 37


In [108]:
# đọc ra training data, label đã lưu trước đó
with open(JSON_ENTITY,'r') as f:
    TRAINING_DATA = json.loads(f.read()) 
    f.close()
    
with open(TOTAL_LABEL_PATH,'r') as f:
    LABEL = json.loads(f.read()) 
    f.close()

print("Total sentences:",len(TRAINING_DATA))
print("Total labels:",len(LABEL),LABEL)

Total sentences: 1487
Total labels: 18 ['PERSON', 'LOC', 'GPE', 'DATE', 'ORG', 'PERCENT', 'QUANTITY', 'TIME', 'LAW', 'MONEY', 'VERB', 'PRODUCT', 'LOCATION', 'PHENOMENA', 'DISEASE', 'ORGANIZATION', 'SEX', 'ANIMAL']


In [109]:
TRAINING_DATA[0]

['"Tôi đứng trong nhà có cảm giác như bị tụt huyết áp, nhìn cốc nước trên bàn thấy rõ bị rung lắc khoảng 10 giây", anh Nguyễn Thanh Hiệp (30 tuổi, ở tầng 20, khu đô thị Times City, quận Hai Bà Trưng) cho biết.',
 {'entities': [[117, 134, 'PERSON'], [179, 183, 'LOC'], [184, 196, 'PERSON']]}]

In [110]:
def format_string(s):
  s = s.replace(' ','')
  s = s.lower()
  return s

# đánh label cho các câu 
# dựa vào hàm chunk của underthesea để lấy ra cấu trúc label của câu là:{word, pos, tag}
# câu là 1 câu hoàn chỉnh, word là từng từ trong câu, pos là thành phần từ đó trong câu và tag là nhãn chunking 
# link refer
# https://github.com/undertheseanlp/underthesea/wiki/M%C3%B4-t%E1%BA%A3-d%E1%BB%AF-li%E1%BB%87u-b%C3%A0i-to%C3%A1n-Chunking
# https://github.com/undertheseanlp/underthesea/wiki/M%C3%B4-t%E1%BA%A3-d%E1%BB%AF-li%E1%BB%87u-b%C3%A0i-to%C3%A1n-POS-Tag

def extract_label(DATA):
  data_train = []
  label_train = []
  for pos in tqdm_notebook(xrange(len(DATA)), desc='build data set'):
    item = DATA[pos]
    text = item[0]
    _chunk = chunk(text) 
    entities = item[1]['entities'] 
    label_sent = []

    for index,tup in enumerate(_chunk): 
      label_sent.append('UNKNOW')
      for en in entities:
      # print(text[en[0]:en[1]],en[2])
        entity = text[en[0]:en[1]]
        if format_string(tup[0]) == format_string(entity):
          label_sent[index] = en[2] 
          break
      
    data_train.append(_chunk) 
    label_train.append(label_sent) 

  return data_train, label_train

data_train, label_train = extract_label(TRAINING_DATA)
data_train[0]
print(label_train[0])

/home/duy/anaconda3/envs/centernet/lib/python3.6/site-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()



['UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'PERSON', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW', 'UNKNOW']


In [111]:
data_train[0]

[('"', 'CH', 'O'),
 ('Tôi', 'P', 'B-NP'),
 ('đứng', 'V', 'B-VP'),
 ('trong', 'E', 'B-PP'),
 ('nhà', 'N', 'B-NP'),
 ('có', 'V', 'B-VP'),
 ('cảm giác', 'N', 'B-NP'),
 ('như', 'C', 'O'),
 ('bị', 'V', 'B-VP'),
 ('tụt', 'V', 'B-VP'),
 ('huyết áp', 'N', 'B-NP'),
 (',', 'CH', 'O'),
 ('nhìn', 'V', 'B-VP'),
 ('cốc', 'N', 'B-NP'),
 ('nước', 'N', 'B-NP'),
 ('trên', 'E', 'B-PP'),
 ('bàn', 'N', 'B-NP'),
 ('thấy', 'V', 'B-VP'),
 ('rõ', 'A', 'B-AP'),
 ('bị', 'V', 'B-VP'),
 ('rung', 'V', 'B-VP'),
 ('lắc', 'V', 'B-VP'),
 ('khoảng', 'N', 'B-NP'),
 ('10', 'M', 'B-NP'),
 ('giây', 'Nu', 'B-NP'),
 ('"', 'CH', 'O'),
 (',', 'CH', 'O'),
 ('anh', 'Nc', 'B-NP'),
 ('Nguyễn Thanh Hiệp', 'Np', 'B-NP'),
 ('(', 'CH', 'O'),
 ('30', 'M', 'B-NP'),
 ('tuổi', 'N', 'B-NP'),
 (',', 'CH', 'O'),
 ('ở', 'E', 'B-PP'),
 ('tầng', 'N', 'B-NP'),
 ('20', 'M', 'B-NP'),
 (',', 'CH', 'O'),
 ('khu', 'N', 'B-NP'),
 ('đô thị', 'N', 'B-NP'),
 ('Times City', 'Np', 'B-NP'),
 (',', 'CH', 'O'),
 ('quận Hai Bà Trưng', 'N', 'B-NP'),
 (')', 'CH',

In [112]:
# sentence to features
X_train = [sent2features(s) for s in data_train] 
y_train = label_train

In [113]:
X_train[0][1]

{'bias': 1.0,
 'word.lower()': 'tôi',
 'word[-3:]': 'Tôi',
 'word.isupper()': False,
 'word.istitle()': True,
 'word.isdigit()': False,
 'postag': 'P',
 'postag[:2]': 'P',
 '-1:word.lower()': '"',
 '-1:word.istitle()': False,
 '-1:word.isupper()': False,
 '-1:postag': 'CH',
 '-1:postag[:2]': 'CH',
 '+1:word.lower()': 'đứng',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'V',
 '+1:postag[:2]': 'V'}

In [114]:
y_train[18]

['UNKNOW',
 'DATE',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'ORG',
 'GPE',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'GPE',
 'UNKNOW',
 'GPE',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'GPE',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'GPE',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'GPE',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'GPE',
 'UNKNOW',
 'GPE',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW',
 'UNKNOW']

In [115]:
import time
# %%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    model_filename='/home/duy/Documents/Duy/NLP/problem_1_nlp/nlp_2/json/' + str(time.time())+ 'nlp_2.h5' 
)

crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

In [116]:
eli5.show_weights(crf, top=10)

From \ To,Animal,DATE,Date,Disease,GPE,LAW,LOC,Location,ORG,Organization,PERSON,Person,Product,TIME,UNKNOW,Verb,phenomena
Animal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.08,0.08,0.0
DATE,0.0,-0.882,0.0,0.0,0.0,0.0,-0.299,0.0,0.446,0.0,0.0,0.0,0.0,0.0,0.188,0.0,0.0
Date,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.16,0.0,0.0,0.0,0.0,0.0,0.08,0.0,0.0
Disease,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GPE,0.0,-0.87,0.0,0.0,-0.395,0.0,0.0,0.0,-0.067,0.0,0.467,0.0,0.0,0.0,0.623,0.237,0.0
LAW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.756,0.0,0.0
LOC,0.0,-0.35,0.0,0.0,-1.194,0.0,0.438,0.0,-0.429,0.0,-0.169,-0.108,0.0,0.0,0.495,0.0,0.0
Location,0.0,0.0,0.0,0.0,-0.152,0.0,-0.131,0.0,0.0,0.0,-0.122,0.0,0.0,0.0,0.06,0.0,0.0
ORG,0.0,-1.654,0.0,0.0,1.24,0.0,-0.229,-0.373,-0.02,0.0,1.047,-0.483,0.0,-0.597,0.193,-0.773,0.0
Organization,0.0,0.0,0.0,0.0,1.215,0.0,0.0,0.0,0.0,0.0,-0.035,0.0,0.0,0.0,-0.067,0.0,0.0


In [123]:
st = TRAINING_DATA[1477][0]
print(st)
# st ="Ngày 2/11, ông Nguyễn Chí Thanh, Giám đốc Công ty BOT Thái Nguyên - Chợ Mới, cho biết, đơn vị đang cấp thẻ miễn, giảm phí cho phương tiện của người dân 101 phường, xã thuộc các huyện Phú Lương, Định Hóa, Đại Từ và TP Thái Nguyên trước khi triển khai thu phí. Dự kiến sẽ có trên 31.000 hộ dân trong diện được miễn giảm phí theo 3 vùng trong bán kính dưới 30km. Bộ Giao thông Vận tải đã quy định khu vực miễn giảm cho người dân trong bán kính trạm thu phí dưới 10km, song chúng tôi mở rộng bán kính đến 30km hỗ trợ người dân"
_c = chunk(st)
y_pred = crf.predict([sent2features(_c)]) 
for index,item in enumerate(_c): 
    if y_pred[0][index] == 'UNKNOW':
        continue
    print(y_train[1477][index])
    print(item, y_pred[0][index])


Ngày 28/4 Tập đoàn Vingroup công bố hoàn tất, chuẩn bị đưa ra thị trường hai mẫu máy thở xâm nhập VFS-410 và VFS-510 phục vụ điều trị bệnh nhân Covid-19 và sử dụng lâu dài trong y tế.  VSmart VFS-410 là bản nâng cấp của phiên bản máy thở đầu tiên VFS-310 do các kỹ sư Vingroup phát triển từ ý tưởng thiết kế của Đại học MIT, còn VSmart VFS-510 nhận chuyển giao, cải tiến từ hãng sản xuất máy thở hàng đầu thế giới Medtronic (Mỹ). Dự kiến lô sản phẩm đầu tiên sẽ xuất xưởng vào ngày 15/5.      VSmart VFS-410 là máy thở xâm nhập dựa trên công nghệ tạo khí bằng tuabin, có tính năng tương đương với các máy thở xâm nhập lưu động cao cấp trên thị trường  Đây là dòng máy được các kỹ sư Việt Nam cải tiến từ thiết kế máy thở không xâm nhập đơn giản của nhóm nghiên cứu trường Đại học MIT.Máy được bổ sung cảm biến giám sát và cảnh báo để duy trì nồng độ oxy, áp suất dương cuối kỳ thở ra, đo được nhịp thở bệnh nhân và tự điều chỉnh để đồng bộ với nhịp thở này. Toàn bộ nguyên lý hoạt động, bo mạch, linh

In [118]:
TRAINING_DATA[18]

['Tối 8/11, Bộ Công an và cảnh sát Anh thông tin\xa0chi tiết về tên tuổi, quê quán của 39 người chết trong container tại hạt Essex, tất cả đều quốc tịch Việt Nam. Nghệ An là tỉnh có nhiều nạn nhân nhất với 21 người, Hà Tĩnh 10 người, Hải Phòng 3 người, Quảng Bình 3 người, Thừa Thiên Huế và Hải Dương mỗi tỉnh một người.',
 {'entities': [[4, 8, 'DATE'],
   [10, 20, 'ORG'],
   [148, 156, 'GPE'],
   [158, 165, 'GPE'],
   [211, 218, 'GPE'],
   [229, 238, 'GPE'],
   [248, 258, 'GPE'],
   [268, 282, 'GPE'],
   [286, 295, 'GPE'],
   [24, 32, 'ORG'],
   [33, 37, 'GPE']]}]

In [119]:
for item in TRAINING_DATA[1418][1]['entities']:
  print(TRAINING_DATA[1418][0][item[0]:item[1]],item)

6/11 [37, 41, 'DATE']
Việt Nam [58, 66, 'GPE']
Anh  [85, 89, 'GPE']


In [120]:
crf.save

AttributeError: 'CRF' object has no attribute 'save'

In [ ]:
crf.model_filename("/content/drive/My Drive/NLP/nlp_problem_2/model")
# load_model(path, custom_objects={'CRF':CRF})

**TRAINING BY SPACY MODEL**


In [ ]:

JSON_FROM_DOCCANO_SPACY = ["/content/drive/My Drive/NLP/nlp_problem_2/json/0-29.json1",
                     "/content/drive/My Drive/NLP/nlp_problem_2/json/30-59.json1",
                     "/content/drive/My Drive/NLP/nlp_problem_2/json/75-100.json1"]
JSON_ENTITY_SPACY = "/content/drive/My Drive/NLP/nlp_problem_2/json/data_spacy.json"
TOTAL_LABEL_SPACY_PATH = "/content/drive/My Drive/NLP/nlp_problem_2/json/label_spacy.json"
MODEL_PATH = "/content/drive/My Drive/NLP/nlp_problem_2/model"

In [ ]:
TRAINING_DATA_SPACY = []
LABEL_SPACY = []
# modify json
for path in JSON_FROM_DOCCANO:
  print(path,end=" ") 
  with open(path) as f:
      TEXTS = f.readlines() 
      print("sentences",len(TEXTS))
      for doc in TEXTS:
          doc = json.loads(doc) 
          entities = [(en[0], en[1], en[2]) for en in doc['labels']]
          [LABEL_SPACY.append(item[2]) for item in entities if item[2] not in LABEL_SPACY]
          if len(entities) == 0:
            continue 
          training_example = (doc['text'], {"entities": entities})
          TRAINING_DATA_SPACY.append(training_example) 
  f.close()

# ghi file json entity
with open(JSON_ENTITY_SPACY,'w') as f:
    f.write(json.dumps(TRAINING_DATA_SPACY))
 
with open(TOTAL_LABEL_SPACY_PATH,'w') as f:
    f.write(json.dumps(LABEL_SPACY))

In [0]:
with open(JSON_ENTITY_SPACY,'r') as f:
    TRAINING_DATA_SPACY = json.loads(f.read()) 

with open(TOTAL_LABEL_SPACY_PATH,'r') as f:
    LABEL_SPACY = json.loads(f.read()) 
    f.close()

nlp = spacy.blank("vi")
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)
[ner.add_label(item) for item in LABEL_SPACY]
print(ner)
len(TRAINING_DATA_SPACY)
LABEL_SPACY

['PERSON',
 'LOC',
 'GPE',
 'DATE',
 'ORG',
 'PERCENT',
 'QUANTITY',
 'TIME',
 'LAW',
 'MONEY']

In [0]:
%%time
# Start the training
nlp.begin_training()

# Loop for 10 iterations
for pos in tqdm_notebook(xrange(50), desc='training'):
    # Shuffle the training data
    random.shuffle(TRAINING_DATA_SPACY)
    losses = {} 
    # Batch the examples and iterate over them
    for batch in spacy.util.minibatch(TRAINING_DATA_SPACY, size=int(len(TRAINING_DATA_SPACY)/100)): 
        texts = [text for text,entities in batch] 
        entities = [entities for text,entities in batch]
        # print(annotations)
        # print(texts)
        # print(entities)
        
        # Update the model 
        nlp.update(texts, entities, losses=losses, drop=0.2)
        
    print(losses)

# Save the model
nlp.to_disk("MODEL_PATH")

{'ner': 58.36445043296329}
{'ner': 122.25501241854019}
{'ner': 162.22320724511724}
{'ner': 168.84208094722828}
{'ner': 191.48822886976362}
{'ner': 215.33410742258417}
{'ner': 143.31360882986743}
{'ner': 165.94797771528914}
{'ner': 155.49978288938004}
{'ner': 149.27261897176564}
{'ner': 136.03358489854492}
{'ner': 108.81837617284401}
{'ner': 105.51298253610516}
{'ner': 105.54501346576215}
{'ner': 99.08328288665192}
{'ner': 113.33532951944191}
{'ner': 95.43729148797904}
{'ner': 164.55994921639837}
{'ner': 115.96881785670709}
{'ner': 140.57259636679422}
{'ner': 167.79372322287762}
{'ner': 113.8386856745044}
{'ner': 98.12283943877358}
{'ner': 91.58924754603616}
{'ner': 131.14292055290565}
{'ner': 96.09369850756892}
{'ner': 83.01182903446102}
{'ner': 112.39895896702312}
{'ner': 94.04318326475541}
{'ner': 123.83422188375721}
{'ner': 84.9397103273133}
{'ner': 90.32658518980683}
{'ner': 78.75666220550727}
{'ner': 62.65151139592079}
{'ner': 112.54221752381142}
{'ner': 85.08109035246034}
{'ner':

In [0]:
TEXT_TEST_PATH = "/content/drive/My Drive/NLP/dataset/Train_Full/My/Chinh_tri_doi_ngoai/XH_VNE_T_ (5040).txt"
TEXT_TRAIN_PATH = "/content/drive/My Drive/NLP/dataset/Train_Full/My/Chinh_tri_doi_ngoai/XH_VNE_T_ (5041).txt"
# test model
content = FileReader(file_path=TEXT_TRAIN_PATH).content()
print(content)
nlp = spacy.load("/content/drive/My Drive/NLP/nlp_problem_2/MODEL_PATH")
text = u"Sau hơn một giờ tham dự nghi lễ Các Đẳng, các giáo dân dọn dẹp lại mộ phần trước khi ra về. Xóm đạo Tha La với khoảng 5.000 giáo dân, được hình thành vào giữa thế kỷ 19. Xóm đạo từng đi vào thơ, nhạc của nhiều tác giả. Nổi bật là bài thơ \"Tha La xóm đạo\" của nhà thơ Vũ Anh Khanh, đã được nhạc sĩ Dũng Chinh phổ nhạc vào năm 1964."
doc = nlp(content) 
for i in doc.ents:
    print(i.text,i.label_)

﻿Những Robinson giữa lòng thành phố
Bồng bềnh trên sông Sài Gòn, ngay gần quận trung tâm TP HCM, là chốt nổi số 1 của trạm biên phòng Nhà Rồng. Trên chiếc sà lan cũ kỹ là 4 chiến sĩ biên phòng sống lẻ loi, thiếu thốn ngay giữa phồn hoa đô thị.
Chốt 1 được giao nhiệm vụ tuần tra mặt nước trên sông, bảo vệ khu vực cảng Nhà Rồng, Ba Son, Tân Cảng và cảng Quân đoàn 4 với chiều dài hơn 5 km. Đây là một nhiệm vụ khó khăn, vì địa bàn này không chỉ là nơi có tới hàng nghìn lượt tàu thuyền trong nước, quốc tế ra vào mà còn tập trung các loại tội phạm như buôn lậu, ma túy, cướp giật. Toàn bộ "quân tướng" Chốt 1 phải túc trực 24/24h . Bất cứ khi nào có lệnh hoặc có dấu hiệu nghi vấn từ các thuyền trên mặt nước là các anh phải lao đến kiểm tra.
Các anh gọi Chốt 1 của mình là "ốc đảo. Bốn người - thiếu tá Nguyễn Văn Út, trung úy Phạm Ngọc Oánh, trung úy Phạm Quốc Thắng và thiếu úy Trần Văn Thành - bốn quê, cả Nam lẫn Bắc nhưng hợp lại đều giống nhau ở mái tóc khô cứng như rễ tre, làn da sạm nắng và

In [0]:
from underthesea import chunk 
chunk("Bộ Công Thương xóa một tổng cục, giảm nhiều đầu mối")

[('Bộ', 'N', 'B-NP'),
 ('Công Thương', 'Np', 'I-NP'),
 ('xóa', 'V', 'B-VP'),
 ('một', 'M', 'B-NP'),
 ('tổng cục', 'N', 'B-NP'),
 (',', 'CH', 'O'),
 ('giảm', 'V', 'B-VP'),
 ('nhiều', 'A', 'B-AP'),
 ('đầu mối', 'N', 'B-NP')]